In [1]:
import numpy as np
import pandas as pd
# Ratio de Calmar
import yfinance as yf

In [2]:
df = yf.Ticker('NQ=F')
df = df.history(start='2000-01-01', end='2024-01-01')

### **Concepto Clave: Ratio de Calmar**

El **Ratio de Calmar** es una métrica utilizada para evaluar el desempeño de una estrategia financiera. Calcula el **retorno anualizado** ajustado por el **riesgo**, donde el riesgo está representado por el **drawdown máximo** (la mayor pérdida acumulada desde un punto máximo hasta un punto mínimo).


$text{Ratio de Calmar} = \frac{\text{Retorno Anualizado}}{\text{Drawdown Máximo Absoluto}}$


- **Retorno Anualizado:** Promedio anual del retorno compuesto de la estrategia.  
- **Drawdown Máximo:** La mayor caída porcentual desde el máximo acumulado al mínimo acumulado.  

Un **alto Ratio de Calmar** indica que la estrategia genera buenos retornos en relación con las pérdidas máximas sufridas.


In [3]:

def backtest_strategy(sma_1, sma_2):
    # Cálculo de SMAs
    df['SMA_50'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_200'] = df['Close'].rolling(window=sma_2).mean()

    # Señales de trading
    df['Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'Signal'] = -1

    # Posición basada en la señal
    df['Position'] = df['Signal'].shift()
    
    # Comprobar que se generaron suficientes señales
    num_signals = df['Position'].diff().abs().sum()  # Número de cambios de señal
    if num_signals < 10:  # Ajusta este umbral según lo que consideres relevante
        return np.nan
    
    # Retornos de la estrategia
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    df.dropna(inplace=True)  # Eliminar filas con NaN
    
    # Asegurarse de que hay suficientes datos
    if len(df) == 0:
        return np.nan
    
    # Calcular drawdown máximo y rendimiento anualizado
    df['Cumulative_Returns'] = (df['Strategy_Returns'] + 1).cumprod()
    df['Peak'] = df['Cumulative_Returns'].cummax()
    df['Drawdown'] = (df['Cumulative_Returns'] - df['Peak']) / df['Peak']
    
    max_drawdown = df['Drawdown'].min()
    total_return = (df['Strategy_Returns'] + 1).prod() - 1
    annualized_return = (1 + total_return) ** (252 / len(df)) - 1

    # Controlar retornos anómalos
    if total_return > 10:  # Si el retorno acumulado es absurdo
        total_return = 10  # Ajustar a un valor razonable
    
    # Evitar división por cero en el Ratio de Calmar
    if max_drawdown == 0 or np.isnan(max_drawdown):
        return np.nan  # Retornar NaN si no hay drawdown o es cero
    
    calmar_ratio = annualized_return / abs(max_drawdown)

    # Limitar valores desproporcionados de Calmar Ratio
    if calmar_ratio > 10:  # Puedes ajustar este umbral
        calmar_ratio = 10  # Limitar a un máximo razonable

    return calmar_ratio

### Optimización mediante Grid Search

In [4]:
best_params = None
best_calmar = -np.inf

for sma_1 in range(10, 100, 10):
    for sma_2 in range(100, 300, 50):
        calmar_ratio = backtest_strategy(sma_1, sma_2)
        if not np.isnan(calmar_ratio):  # Evitar comparar con NaN
            if calmar_ratio > best_calmar:
                best_calmar = calmar_ratio
                best_params = (sma_1, sma_2)

print(f'Los mejores parámetros son: {best_params} con un Ratio de Calmar de {best_calmar:.2f}')

Los mejores parámetros son: (90, 150) con un Ratio de Calmar de 0.54
